In [34]:
# Install gym
! pip install gym
# Install neurogym
! git clone https://github.com/gyyang/neurogym.git
%cd neurogym/
! pip install -e .
!pip install shimmy>=0.2.1
!pip install stable-baselines3
!pip install torchrl


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Cloning into 'neurogym'...
remote: Enumerating objects: 11100, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 11100 (delta 132), reused 234 (delta 115), pack-reused 10827
Receiving objects: 100% (11100/11100), 8.20 MiB | 742.00 KiB/s, done.
Resolving deltas: 100% (8318/8318), done.
/home/maria/Neuromatch2024/neurogym/neurogym/neurogym/neurogym/neurogym
Obtaining file:///home/maria/Neuromatch2024/neurogym/neurogym/neurogym/neurogym/neurogym
  Preparing metadata (setup.py) ... done
  Attempting uninstall: neurogym
    Found existing installation: neurogym 0.0.2
    Uninstalling neurogym-0.0.2:
      Successfully uninstalled neurogym-0.0.2
  Running setup.py develop for neurogym

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependen

Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0a2
    Uninstalling gymnasium-1.0.0a2:
      Successfully uninstalled gymnasium-1.0.0a2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shimmy 2.0.0 requires gymnasium>=1.0.0a1, but you have gymnasium 0.29.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:00:01m

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import numpy as np
import torch
import torch.nn as nn

import neurogym as ngym

# Environment
task = 'PerceptualDecisionMaking-v0'
kwargs = {'dt': 100}
seq_len = 100

# Make supervised dataset
dataset = ngym.Dataset(task, env_kwargs=kwargs, batch_size=16,
                       seq_len=seq_len)
env = dataset.env
ob_size = env.observation_space.shape[0]
act_size = env.action_space.n

In [14]:
print(ob_size)

3


In [15]:
dataset()

(array([[[1.        , 0.7187755 , 0.1145535 ],
         [1.        , 0.41953516, 0.5228772 ],
         [1.        , 0.5432717 , 0.5867997 ],
         ...,
         [1.        , 0.48829374, 0.47994006],
         [1.        , 0.51849645, 0.6885914 ],
         [1.        , 0.39576638, 0.44281998]],
 
        [[1.        , 0.6804584 , 0.22889833],
         [1.        , 0.40111068, 0.6894966 ],
         [1.        , 0.45901948, 0.42352992],
         ...,
         [1.        , 0.3700665 , 0.5051325 ],
         [1.        , 0.4950613 , 0.4118519 ],
         [1.        , 0.4699116 , 0.45070922]],
 
        [[1.        , 0.6649615 , 0.08556549],
         [1.        , 0.3783346 , 0.5691897 ],
         [1.        , 0.4991771 , 0.5272677 ],
         ...,
         [1.        , 0.5430453 , 0.3417082 ],
         [1.        , 0.6089853 , 0.47484654],
         [1.        , 0.40430358, 0.60065967]],
 
        ...,
 
        [[1.        , 0.51592994, 0.479775  ],
         [1.        , 0.6651389 , 0.29275

In [13]:
class Net(nn.Module):
    def __init__(self, num_h):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(ob_size, num_h)
        self.linear = nn.Linear(num_h, act_size)

    def forward(self, x):
        out, hidden = self.lstm(x)
        x = self.linear(out)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net(num_h=64).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)

running_loss = 0.0
for i in range(2000):
    inputs, labels = dataset()
    inputs = torch.from_numpy(inputs).type(torch.float).to(device)
    labels = torch.from_numpy(labels.flatten()).type(torch.long).to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)

    loss = criterion(outputs.view(-1, act_size), labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 200 == 199:
        print('{:d} loss: {:0.5f}'.format(i + 1, running_loss / 200))
        running_loss = 0.0

print('Finished Training')

200 loss: 0.10047
400 loss: 0.02951
600 loss: 0.01366
800 loss: 0.01186
1000 loss: 0.01170
1200 loss: 0.01160
1400 loss: 0.01151
1600 loss: 0.01071
1800 loss: 0.01073
2000 loss: 0.01078
Finished Training


In [ ]:
import gym
import neurogym as ngym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
import numpy as np


# Create a custom multi-task environment
class MultiTaskEnv(gym.Env):
    def __init__(self, task1_id, task2_id, switch_interval):
        super(MultiTaskEnv, self).__init__()
        self.task1 = ngym.make(task1_id)
        self.task2 = ngym.make(task2_id)
        self.switch_interval = switch_interval
        self.current_step = 0

        # Assuming both tasks have the same observation and action spaces
        self.observation_space = self.task1.observation_space
        self.action_space = self.task1.action_space

    def reset(self):
        self.current_step = 0
        self.current_task = self.task1 if self.current_step // self.switch_interval % 2 == 0 else self.task2
        return self.current_task.reset()

    def step(self, action):
        self.current_step += 1
        if self.current_step // self.switch_interval % 2 == 0:
            self.current_task = self.task1
        else:
            self.current_task = self.task2
        obs, reward, done, info = self.current_task.step(action)
        if done:
            obs = self.current_task.reset()
        return obs, reward, done, info


# Define the custom MLP policy
class MLPPolicy(nn.Module):
    def __init__(self, obs_space, act_space, hidden_sizes=[128, 128], activation=nn.ReLU):
        super(MLPPolicy, self).__init__()
        self.obs_space = obs_space
        self.act_space = act_space

        layers = []
        input_size = obs_space.shape[0]
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(activation())
            input_size = hidden_size
        layers.append(nn.Linear(input_size, act_space.n))

        self.policy_net = nn.Sequential(*layers)
        self.value_net = nn.Sequential(
            nn.Linear(obs_space.shape[0], hidden_sizes[0]),
            activation(),
            nn.Linear(hidden_sizes[0], 1)
        )

    def forward(self, x):
        policy_logits = self.policy_net(x)
        value = self.value_net(x)
        return policy_logits, value

    def act(self, x):
        logits, value = self.forward(x)
        probs = torch.softmax(logits, dim=-1)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action, log_prob, value


# Define the PPO agent
class PPOAgent:
    def __init__(self, policy, lr=3e-4, gamma=0.99, k_epochs=4, eps_clip=0.2, device='cuda'):
        self.policy = policy.to(device)
        self.optimizer = optim.Adam(policy.parameters(), lr=lr)
        self.gamma = gamma
        self.k_epochs = k_epochs
        self.eps_clip = eps_clip
        self.device = device

    def compute_returns(self, rewards, masks, values, next_value):
        returns = []
        R = next_value
        for step in reversed(range(len(rewards))):
            R = rewards[step] + self.gamma * R * masks[step]
            returns.insert(0, R)
        return returns

    def update(self, memory):
        states, actions, log_probs, rewards, masks, values = memory

        states = torch.stack(states).to(self.device)
        actions = torch.tensor(actions).to(self.device)
        log_probs = torch.tensor(log_probs).to(self.device)
        values = torch.stack(values).to(self.device)

        next_value = self.policy(states[-1].unsqueeze(0))[1]
        returns = self.compute_returns(rewards, masks, values, next_value)
        returns = torch.tensor(returns).detach().to(self.device)

        old_states = states.detach()
        old_actions = actions.detach()
        old_log_probs = log_probs.detach()

        for _ in range(self.k_epochs):
            logits, state_values = self.policy(old_states)
            dist = Categorical(torch.softmax(logits, dim=-1))
            new_log_probs = dist.log_prob(old_actions)
            entropy = dist.entropy().mean()

            advantages = returns - state_values.squeeze().detach()
            ratio = torch.exp(new_log_probs - old_log_probs)
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            actor_loss = -torch.min(surr1, surr2).mean()
            critic_loss = nn.MSELoss()(state_values.squeeze(), returns)

            loss = 0.5 * critic_loss + actor_loss - 0.01 * entropy

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


# Training loop
def train(env, agent, num_episodes=1000, batch_size=32):
    memory = deque(maxlen=batch_size)
    episode_rewards = []

    for episode in range(num_episodes):
        state = env.reset()
        state = torch.FloatTensor(state).to(agent.device).unsqueeze(0)
        episode_reward = 0

        for t in range(1, 10000):
            env.reset()
            action, log_prob, value = agent.policy.act(state)
            action = action.item()  # Convert action tensor to scalar
            next_state, reward, done, _ = env.step(0)
            next_state = torch.FloatTensor(next_state).to(agent.device).unsqueeze(0)

            memory.append((state, action, log_prob.item(), reward, 1 - int(done), value))
            episode_reward += reward

            if len(memory) == batch_size:
                states, actions, log_probs, rewards, masks, values = zip(*memory)
                agent.update((states, actions, log_probs, rewards, masks, values))
                memory.clear()

            state = next_state
            if done:
                break

        episode_rewards.append(episode_reward)
        if (episode + 1) % 10 == 0:
            print(f"Episode {episode + 1}, Reward: {episode_reward}")

    return episode_rewards


# Main execution
task1_id = 'PerceptualDecisionMaking-v0'
task2_id = 'ReachingDelayResponse-v0'
switch_interval = 100
env = MultiTaskEnv(task1_id, task2_id, switch_interval)

obs_space = env.observation_space
act_space = env.action_space

policy = MLPPolicy(obs_space, act_space)
agent = PPOAgent(policy)

episode_rewards = train(env, agent)

# Evaluate the model
state = env.reset()
state = torch.FloatTensor(state).to(agent.device).unsqueeze(0)
for _ in range(1000):
    action, _, _ = policy.act(state)
    action = action.item()  # Convert action tensor to scalar
    next_state, reward, done, _ = env.step(0)
    next_state = torch.FloatTensor(next_state).to(agent.device).unsqueeze(0)
    env.render()
    state = next_state
    if done:
        state = env.reset()
        state = torch.FloatTensor(state).to(agent.device).unsqueeze(0)

env.close()
